# **Template OP on salish**

In [1]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4, AdvectionRK4_3D

sys.path.append('/Users/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels import DeleteParticle, Buoyancy
from OP_functions import *

In [2]:
# Define paths
local = 1 #Set to 0 when working on server
paths = path(local)

## Definitions

In [3]:
start = datetime(2018, 8, 23) #Start date
# Set Time length [days] and timestep [seconds]
length = 1 
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 6 # number of deploying locations
n = 100 # 1000   # number of particles per location
dmin = [0,0,0,0,0,70] #minimum depth
dd = 20 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [4]:
Dat=xr.open_dataset('/Users/jvalenti/MOAD/SSC_masks/coordinates_seagrid_SalishSea201702.nc',decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]

In [5]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
clon, clat = [-123.901172,-125.155849,-123.207648,-122.427508,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,47.622403,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [6]:
#Set start date time and the name of the output file
name = 'testlocal' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
paths = path()
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/Users/jvalenti/MOAD/analysis-jose/notebooks/results/testlocal20180823_1n_20180824_1n.nc


## Simulation

In [7]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

In [8]:
# filenames2,variables,dimensions=filename_set(start,length,varlist,0)
# from paramiko import SSHClient
# from scp import SCPClient

# duration = timedelta(days=length)
# ssh = SSHClient()
# ssh.load_system_host_keys()
# ssh.connect('salish.eos.ubc.ca')
# for day in range(duration.days):
# # # SCPCLient takes a paramiko transport as an argument
#     scp = SCPClient(ssh.get_transport())
#     date=start + timedelta(days=day)
#     directorio='/Users/jvalenti/MOAD/data/'+date.strftime("%d%b%y").lower()+'/'
#     os.mkdir(directorio)
# # #scp.put('test.txt', 'test2.txt')
#     scp.get(filenames2['U']['data']+filenames2['V']['data']+filenames2['W']['data'],local_path=directorio)

# # # Uploading the 'test' directory with its content in the
# # # '/home/user/dump' remote directory
# # #scp.put('test', recursive=True, remote_path='/home/user/dump')

# scp.close()

In [ ]:
# #Load Salish output as fields
field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, JITParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

#p_flt = pset.Kernel(Buoyancy) #uncomment if you want to use additional kernels, also change the value of pset.Kernel to "AdvectionRK4_3D + p_flt"
#pset.computeTimeChunk(allow_time_extrapolation=1)
pset.execute(
    pset.Kernel(AdvectionRK4_3D), runtime=duration, dt=dt,
    output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
    recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
)

         It will be opened with no decoding. Filling values might be wrongly parsed.
INFO: Compiled ArrayJITParticleAdvectionRK4_3D ==> /var/folders/y1/__7_22kx3n9_rh91qk2htyc00000gn/T/parcels-501/lib381e200181d14227f7a2e6f8dac82508_0.so
INFO: Temporary output files are stored in /Users/jvalenti/MOAD/analysis-jose/notebooks/results/out-RXALRQAS.
INFO: You can use "parcels_convert_npydir_to_netcdf /Users/jvalenti/MOAD/analysis-jose/notebooks/results/out-RXALRQAS" to convert these to a NetCDF file during the run.
100% (86400.0 of 86400.0) |##############| Elapsed Time: 0:00:06 Time:  0:00:06
